In [1]:
import os

os.environ["GROQ_API_KEY"] = "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"

In [2]:
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END, add_messages
from pydantic import BaseModel
from typing import List
from langgraph.checkpoint.memory import MemorySaver
from langchain.schema import HumanMessage, AIMessage

# Initialize memory-based checkpointing
memory = MemorySaver()

# Define State Class with add_messages for conversation tracking
class ChatState(BaseModel):
    messages: List[HumanMessage | AIMessage] = add_messages()  # Automatically tracks conversation

# Initialize LangChain components
llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

# LLM Node to handle conversation
def llm_node(state: ChatState) -> ChatState:
    """Generate response using LLM while maintaining conversation history."""
    
    # Extract past conversation messages and format as history
    history_text = "\n".join([msg.content for msg in state.messages])
    
    prompt = f"""
    Previous conversation:
    {history_text}
    
    Answer the current question based on the provided context.
    """
    
    # Generate response using LLM
    response = llm.invoke(prompt).content
    
    # Update conversation history
    state.messages.append(AIMessage(content=response))
    
    return {"messages": state.messages}

# Build the LangGraph workflow
graph = StateGraph(ChatState)
graph.add_node("llm_node", llm_node)

# Define edges
graph.add_edge(START, "llm_node")
graph.add_edge("llm_node", END)  # Ensure LLM output leads to END

# Compile Graph with checkpointing
compiled_graph = graph.compile(checkpointer=memory)
    


# from IPython.display import Image, display
# from langchain_core.runnables.graph import MermaidDrawMethod

# display(
#     Image(
#         compiled_graph.get_graph().draw_mermaid_png(
#             draw_method=MermaidDrawMethod.API,
#         )
#     )
# )


In [3]:
initial_state = ChatState(
    messages=[HumanMessage("can you find where i work currently in my resume?")]
)
config = {"configurable": {"thread_id": "1"}}
result = compiled_graph.invoke(initial_state, config=config)
result

{'messages': [HumanMessage(content='can you find where i work currently in my resume?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Based on the previous conversation, I would need to access your resume to find out where you currently work. However, I am a text-based AI and do not have the ability to access or view files or documents. If you can provide the relevant information from your resume, I would be happy to help with any questions or concerns you have.', additional_kwargs={}, response_metadata={})]}

In [4]:
result

{'messages': [HumanMessage(content='can you find where i work currently in my resume?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Based on the previous conversation, I would need to access your resume to find out where you currently work. However, I am a text-based AI and do not have the ability to access or view files or documents. If you can provide the relevant information from your resume, I would be happy to help with any questions or concerns you have.', additional_kwargs={}, response_metadata={})]}

In [5]:
state = ChatState(
    query="my name is yegyanathan",
)
config = {"configurable": {"thread_id": "1"}}
result = compiled_graph.invoke(state, config=config)
print(result['response'])

/Users/yegyanathanv/anaconda3/envs/chatapp/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `list[Union[HumanMessage, AIMessage]]` but got `partial` with value `functools.partial(<functi...essages at 0x11dcb7600>)` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


InvalidUpdateError: Must write to at least one of ['messages']

In [ ]:
state = ChatState(
    query="what is my name",
)
config = {"configurable": {"thread_id": "1"}}
result = compiled_graph.invoke(state, config=config)
print(result['response'])

In [ ]:
compiled_graph.aget_state_history(config)

In [7]:
state = ChatState(
    query="what is my name again",
)
config = {"configurable": {"thread_id": "1"}}
result = compiled_graph.invoke(state, config=config)
print(result['response'])

/Users/yegyanathanv/anaconda3/envs/chatapp/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `list[Union[HumanMessage, AIMessage]]` but got `partial` with value `functools.partial(<functi...essages at 0x11dcb7600>)` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


InvalidUpdateError: Must write to at least one of ['messages']

In [6]:
state = ChatState(
    query="what is my name again",
)
config = {"configurable": {"thread_id": "2"}}
result = compiled_graph.invoke(state, config=config)
print(result['response'])

/Users/yegyanathanv/anaconda3/envs/chatapp/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `list[Union[HumanMessage, AIMessage]]` but got `partial` with value `functools.partial(<functi...essages at 0x11dcb7600>)` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


InvalidUpdateError: Must write to at least one of ['messages']

In [10]:
config = {"configurable": {"thread_id": "1"}}
state = compiled_graph.get_state(config).values["messages"]

In [11]:
state

[HumanMessage(content='can you find where i work currently in my resume?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Based on the previous conversation, I would need to access your resume to find out where you currently work. However, I am a text-based AI and do not have the ability to access or view files or documents. If you can provide the relevant information from your resume, I would be happy to help with any questions or concerns you have.', additional_kwargs={}, response_metadata={})]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]}).messages

In [12]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph

builder = StateGraph(int)
builder.add_node("add_one", lambda x: x + 1)
builder.set_entry_point("add_one")
builder.set_finish_point("add_one")
conn = sqlite3.connect("checkpoints.sqlite")
memory = SqliteSaver(conn)
graph = builder.compile(checkpointer=memory)

# config = {"configurable": {"thread_id": "1"}}
# graph.get_state(config)
# result = graph.invoke(3, config)
# graph.get_state(config)

In [13]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)  # List of table names

[('checkpoints',), ('writes',)]


In [9]:
# Get table schema
table_name = "checkpoints"
cursor.execute(f"PRAGMA table_info({table_name})")

# Fetch and print results
columns = cursor.fetchall()
for col in columns:
    print(col)

conn.close()


(0, 'thread_id', 'TEXT', 1, None, 1)
(1, 'checkpoint_ns', 'TEXT', 1, "''", 2)
(2, 'checkpoint_id', 'TEXT', 1, None, 3)
(3, 'parent_checkpoint_id', 'TEXT', 0, None, 0)
(4, 'type', 'TEXT', 0, None, 0)
(5, 'checkpoint', 'BLOB', 0, None, 0)
(6, 'metadata', 'BLOB', 0, None, 0)


In [17]:
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT thread_id FROM checkpoints;")
thread_ids = [tid[0] for tid in cursor.fetchall()]
thread_ids

['1']

In [ ]:
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("messages")
])

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("messages"),
    ("human", "{prompt}")
])

# LLM Node to handle conversation
def llm_node(state: ChatState) -> ChatState:
    """Generate response using LLM while maintaining conversation history."""

    context_parts = []  # Collect available contexts dynamically
    
    if state.rag_context:
        context_parts.append(f"RAG Context:\n{state.rag_context}")
    if state.web_context:
        context_parts.append(f"Web Context:\n{state.web_context}")
    
    # Join all available parts with spacing
    context_str = "\n\n".join(context_parts) if context_parts else "No additional context available."
    
    # Final formatted string
    prompt = f"User Query:\n{state.curr_query}\n\n{context_str}"
    
    # Extract past conversation messages and format as history
    prompt = prompt_template.format(messages=state.messages, prompt=prompt)
    
    # Generate response using LLM
    response = llm.invoke(prompt).content
    
    # Update conversation history
    state.messages.append(AIMessage(content=response))
    
    return {"messages": state.messages}

In [ ]:
import os
import sqlite3
import argparse

from pydantic import BaseModel
from typing import Optional, List
from langchain_groq import ChatGroq
from langchain_milvus import Milvus
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain.schema import HumanMessage, AIMessage
from langchain_community.tools import TavilySearchResults
from langchain_huggingface import HuggingFaceEmbeddings
from langgraph.graph import StateGraph, START, END, add_messages
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

os.environ["GROQ_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = "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"

########## Defaults ##########

TEMPERATURE=0.5
LLM_MODEL_NAME="mixtral-8x7b-32768"
EMBED_MODEL_NAME="sentence-transformers/all-MiniLM-L6-v2"
MILVUS_URI="./milvus_example.db"

########## Components ##########

print("Setting up LLM...")
llm = ChatGroq(temperature=TEMPERATURE, model_name=LLM_MODEL_NAME)

print("Setting up embed model...")
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)

print("Setting up Milvus vector DB...")
vector_db = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": MILVUS_URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    auto_id=True
)

print("Setting up Tavily tool...")
web_search = TavilySearchResults(
    max_results=1,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True
)